# Excel Sheet Metadata Extraction - tsDTA Processor

**Purpose**: Extract sheet metadata (names, indices, categories) from tsDTA Excel documents stored in md_dta_history table.

**Input**: md_dta_history table (filtered for tsDTA Excel documents with READY_FOR_PROCESSING status)

**Output**: md_dta_excel_file_raw table (one row per sheet - metadata only, NO data content)

**Processing**: Batch processing with conditional OPTIMIZE for scalability

**Filters Applied**:
- Tags contain "tsDTA"
- Active = true
- Current status = "READY_FOR_PROCESSING"
- File extension matches Excel formats (.xls, .xlsx, .xlsm, .xlsb, .xltx, .xltm)

**Optimizations**: 
- Z-ORDER by (document_id, sheet_category) for fast queries
- Conditional OPTIMIZE to prevent small file accumulation
- Batch processing (no circular CDF dependencies)


In [ ]:
# Cell 1: Imports
import os
import json
import base64
import uuid
import pandas as pd
from io import BytesIO
from datetime import datetime
from pyspark.sql import functions as F
from pyspark.sql.functions import lit, current_timestamp
from delta.tables import DeltaTable

from clinical_data_standards_framework.utils import (
    get_excel_sheets_schema,
    get_document_status_schema,
    categorize_sheet_name,
    extract_version_history_metadata,
    save_with_audit
)


In [ ]:
# Cell 2: Get Configuration from Setup Task
globals_dict = json.loads(dbutils.jobs.taskValues.get(taskKey="setup", key="globals"))
pipeline_config = json.loads(dbutils.jobs.taskValues.get(taskKey="setup", key="pipeline_config"))
created_by_principal = dbutils.jobs.taskValues.get(taskKey="setup", key="created_by_principal")
databricks_job_id = dbutils.jobs.taskValues.get(taskKey="setup", key="databricks_job_id")
databricks_job_name = dbutils.jobs.taskValues.get(taskKey="setup", key="databricks_job_name")
databricks_run_id = dbutils.jobs.taskValues.get(taskKey="setup", key="databricks_run_id")

# Extract needed values from globals
catalog = globals_dict['catalog']
schema = globals_dict['bronze_schema']

print(f"Catalog: {catalog}, Schema: {schema}")
print(f"Created by: {created_by_principal}")
print(f"Job ID: {databricks_job_id}, Job Name: {databricks_job_name}, Run ID: {databricks_run_id}")


In [ ]:
# Cell 3: Extract Pipeline-Specific Config
source_config = pipeline_config['xls']['standard']['source']
output_config = pipeline_config['xls']['standard']['output']

source_table = source_config['source_table']
filter_tags = source_config['filter_tags']
filter_active = source_config['filter_active']
filter_status = source_config['filter_status']
excel_sheets_table = output_config['excel_sheets_table']
excel_sheets_table_description = output_config.get('excel_sheets_table_description', '')

# Construct full table names
source_table_full_name = f"{catalog}.{schema}.{source_table}"
excel_sheets_table_full_name = f"{catalog}.{schema}.{excel_sheets_table}"
status_table_full_name = f"{catalog}.{schema}.md_document_status"

print(f"Source table: {source_table_full_name}")
print(f"Output table: {excel_sheets_table_full_name}")
print(f"Status table: {status_table_full_name}")
print(f"Filter: tags={filter_tags}, active={filter_active}, status={filter_status}")


In [ ]:
# Cell 4: Read Documents - BATCH MODE
# Read from md_dta_history table (batch read, not streaming)
# This avoids circular dependencies when updating the same table
df_source = spark.read.format("delta").table(source_table_full_name)

# Filter for tsDTA Excel documents that are ready for processing
# File pattern: *.xls* matches .xls, .xlsx, .xlsm, .xlsb, .xltx, .xltm
df_filtered = df_source.filter(
    (F.array_contains(F.col("document_tags"), filter_tags[0])) &
    (F.col("active") == filter_active) &
    (F.col("current_status") == filter_status) &
    (F.col("file_extension").rlike("^\\.(xls|xlsx|xlsm|xlsb|xltx|xltm)$"))
)

# Count documents to process
doc_count = df_filtered.count()
print(f"{'='*80}")
print(f"Documents to process: {doc_count}")
print(f"{'='*80}")

if doc_count == 0:
    print("⚠ No documents to process.")
    print(f"Checking source table {source_table_full_name}...")
    
    total_count = df_source.count()
    print(f"  Total records in source: {total_count}")
    
    tag_count = df_source.filter(F.array_contains(F.col("document_tags"), filter_tags[0])).count()
    print(f"  Records with tag '{filter_tags[0]}': {tag_count}")
    
    active_count = df_source.filter(F.col("active") == filter_active).count()
    print(f"  Records with active={filter_active}: {active_count}")
    
    status_count = df_source.filter(F.col("current_status") == filter_status).count()
    print(f"  Records with status='{filter_status}': {status_count}")
    
    excel_count = df_source.filter(F.col("file_extension").rlike("^\\.(xls|xlsx|xlsm|xlsb|xltx|xltm)$")).count()
    print(f"  Records with Excel extensions: {excel_count}")
    
    # Set task value to signal downstream tasks to skip
    dbutils.jobs.taskValues.set(key="documents_found", value="false")
    dbutils.jobs.taskValues.set(key="document_count", value="0")
    
    print("\n⏭️  Setting task value 'documents_found' = false")
    print("   Downstream tasks will be skipped.")
    dbutils.notebook.exit("No documents to process - downstream tasks will be skipped")

# Set task value indicating documents were found
dbutils.jobs.taskValues.set(key="documents_found", value="true")
dbutils.jobs.taskValues.set(key="document_count", value=str(doc_count))
print(f"\n✓ Setting task value 'documents_found' = true")
print(f"  Document count: {doc_count}")


In [ ]:
# Cell 5: Process Excel Files - Batch Function
def process_excel_batch(batch_df, batch_id):
    """
    Process a batch of Excel documents:
    - Decode base64 content
    - Extract sheet metadata (names, indices, categories)
    - Save to md_dta_excel_file_raw table (with audit columns)
    - Update document status in both status and manifest tables
    """
    rows = batch_df.select('document_id', 'extracted_path', 'content_base64').collect()
    print(f'=' * 80)
    print(f'Processing batch {batch_id}, num Excel files = {len(rows)}')
    print(f'=' * 80)
    
    all_sheet_records = []
    all_status_records = []
    document_updates = []
    
    for row in rows:
        document_id = row['document_id']
        extracted_path = row['extracted_path']
        content_b64 = row['content_base64']
        
        start_time = datetime.now()
        
        try:
            # Status: STARTED
            all_status_records.append({
                'document_id': document_id,
                'status': 'EXCEL_EXTRACTION_STARTED',
                'status_timestamp': start_time,
                'error_message': None,
                'processing_duration_seconds': None,
                'retry_count': 0,
                'processing_metadata': json.dumps({'source_path': extracted_path})
            })
            
            # Decode base64 content
            content_binary = base64.b64decode(content_b64)
            
            # Read Excel file with pandas - detect correct engine based on file extension
            excel_file = BytesIO(content_binary)
            
            # Detect file extension and choose correct engine
            file_ext = os.path.splitext(extracted_path.lower())[1]
            
            if file_ext in ['.xls', '.xlt']:
                # Old Excel format requires xlrd
                xl = pd.ExcelFile(excel_file, engine='xlrd')
            else:
                # New Excel format (.xlsx, .xlsm, .xlsb, .xltx, .xltm)
                xl = pd.ExcelFile(excel_file, engine='openpyxl')
            
            print(f'  Document: {document_id}')
            print(f'    Path: {extracted_path}')
            print(f'    Sheets: {len(xl.sheet_names)}')
            
            # Extract Version History metadata once per Excel file
            version_metadata = extract_version_history_metadata(content_binary, file_ext)
            trial_id = version_metadata.get("trial_id", "UNKNOWN")
            data_stream_type = version_metadata.get("data_stream_type", "UNKNOWN")
            data_provider_name = version_metadata.get("data_provider_name", "UNKNOWN")
            
            print(f'    Version History: trial_id={trial_id}, stream={data_stream_type}, provider={data_provider_name}')
            
            # Extract each sheet metadata with category
            for sheet_idx, sheet_name in enumerate(xl.sheet_names):
                # Categorize sheet using helper function
                sheet_category = categorize_sheet_name(sheet_name)
                
                sheet_record = {
                    'document_id': str(uuid.uuid4()),  # Unique sheet ID
                    'parent_document_id': document_id,  # Reference to parent Excel file
                    'source_file_path': extracted_path,
                    'sheet_name': sheet_name,
                    'sheet_index': sheet_idx,
                    'sheet_category': sheet_category,
                    'trial_id': trial_id,
                    'data_stream_type': data_stream_type,
                    'data_provider_name': data_provider_name
                }
                all_sheet_records.append(sheet_record)
                
                print(f'      [{sheet_idx}] {sheet_name} → {sheet_category}')
            
            # Calculate duration
            end_time = datetime.now()
            duration = (end_time - start_time).total_seconds()
            
            # Status: COMPLETED
            all_status_records.append({
                'document_id': document_id,
                'status': 'EXCEL_EXTRACTION_COMPLETED',
                'status_timestamp': end_time,
                'error_message': None,
                'processing_duration_seconds': duration,
                'retry_count': 0,
                'processing_metadata': json.dumps({
                    'sheets_extracted': len(xl.sheet_names),
                    'source_path': extracted_path
                })
            })
            
            # Mark for status update in trial_data_history
            document_updates.append({
                'document_id': document_id,
                'new_status': 'EXCEL_EXTRACTION_COMPLETED',
                'new_status_timestamp': end_time
            })
            
            print(f'    ✓ SUCCESS - {duration:.2f}s')
            
        except Exception as e:
            # Status: ERROR
            error_time = datetime.now()
            duration = (error_time - start_time).total_seconds()
            
            all_status_records.append({
                'document_id': document_id,
                'status': 'EXCEL_EXTRACTION_ERROR',
                'status_timestamp': error_time,
                'error_message': str(e),
                'processing_duration_seconds': duration,
                'retry_count': 0,
                'processing_metadata': json.dumps({'source_path': extracted_path})
            })
            
            document_updates.append({
                'document_id': document_id,
                'new_status': 'EXCEL_EXTRACTION_ERROR',
                'new_status_timestamp': error_time
            })
            
            print(f'    ✗ ERROR - {str(e)}')
    
    # Write results with audit columns
    if all_sheet_records:
        print(f'\nSaving {len(all_sheet_records)} sheets to {excel_sheets_table_full_name}...')
        sheets_df = spark.createDataFrame(all_sheet_records, schema=get_excel_sheets_schema())
        
        # save_with_audit automatically adds audit columns
        save_with_audit(
            df=sheets_df,
            table_name=excel_sheets_table_full_name,
            created_by_principal=created_by_principal,
            databricks_job_id=databricks_job_id,
            databricks_job_name=databricks_job_name,
            databricks_run_id=databricks_run_id,
            mode='append'
        )
        print(f'✓ Saved {len(all_sheet_records)} sheets with audit columns')
    
    if all_status_records:
        print(f'\nAppending {len(all_status_records)} status records...')
        status_df = spark.createDataFrame(all_status_records, schema=get_document_status_schema())
        
        # save_with_audit automatically adds audit columns
        save_with_audit(
            df=status_df,
            table_name=status_table_full_name,
            created_by_principal=created_by_principal,
            databricks_job_id=databricks_job_id,
            databricks_job_name=databricks_job_name,
            databricks_run_id=databricks_run_id,
            mode='append'
        )
        print(f'✓ Appended {len(all_status_records)} status records with audit columns')
    
    if document_updates:
        print(f'\nUpdating {len(document_updates)} documents in manifest...')
        updates_df = spark.createDataFrame(document_updates)
        delta_table = DeltaTable.forName(spark, source_table_full_name)
        
        delta_table.alias("target").merge(
            updates_df.alias("updates"),
            "target.document_id = updates.document_id"
        ).whenMatchedUpdate(set={
            "current_status": "updates.new_status",
            "current_status_timestamp": "updates.new_status_timestamp",
            "last_updated_by_principal": lit(created_by_principal),
            "last_updated_ts": current_timestamp(),
            "databricks_job_id": lit(databricks_job_id),
            "databricks_run_id": lit(databricks_run_id)
        }).execute()
        
        print(f'✓ Updated {len(document_updates)} documents')
    
    print(f'=' * 80)


In [ ]:
# Cell 6: Process Documents in Batch
print(f"{'='*80}")
print(f"Starting batch processing...")
print(f"{'='*80}")

# Process all filtered documents
process_excel_batch(df_filtered, batch_id=0)

print(f"{'='*80}")
print(f"✓ Batch processing completed!")
print(f"{'='*80}")


In [ ]:
# Cell 7: Conditional OPTIMIZE - md_dta_excel_file_raw table
# Only run OPTIMIZE when conditions are met to save compute resources

# Get table stats
table_stats = spark.sql(f"DESCRIBE DETAIL {excel_sheets_table_full_name}").first()
num_files = table_stats.numFiles
total_size_gb = table_stats.sizeInBytes / (1024**3)
avg_file_size_mb = (total_size_gb * 1024) / num_files if num_files > 0 else 0

print(f"\n{'='*80}")
print(f"TABLE STATS: {excel_sheets_table_full_name}")
print(f"{'='*80}")
print(f"Sheets processed: {doc_count}")
print(f"Total files: {num_files}")
print(f"Total size: {total_size_gb:.2f} GB")
print(f"Avg file size: {avg_file_size_mb:.2f} MB")

# Determine if OPTIMIZE is needed
should_optimize = False
reason = ""

if doc_count > 100:  # Condition 1: Large batch
    should_optimize = True
    reason = f"Large batch processed ({doc_count} documents)"
elif num_files > 200 and avg_file_size_mb < 100:  # Condition 2: Small file problem
    should_optimize = True
    reason = f"Small file problem ({num_files} files, avg {avg_file_size_mb:.2f}MB)"
elif num_files > 500:  # Condition 3: Too many files
    should_optimize = True
    reason = f"Too many files ({num_files})"

if should_optimize:
    print(f"\n🔧 OPTIMIZE TRIGGERED: {reason}")
    print(f"{'='*80}")
    
    start_time = datetime.now()
    
    # Run OPTIMIZE with Z-ORDER
    spark.sql(f"""
        OPTIMIZE {excel_sheets_table_full_name}
        ZORDER BY (document_id, sheet_category)
    """)
    
    # Get stats after optimization
    table_stats_after = spark.sql(f"DESCRIBE DETAIL {excel_sheets_table_full_name}").first()
    num_files_after = table_stats_after.numFiles
    
    duration = (datetime.now() - start_time).total_seconds()
    
    print(f"✓ OPTIMIZE completed in {duration:.1f} seconds")
    print(f"  Files: {num_files} → {num_files_after} ({num_files - num_files_after} merged)")
    print(f"{'='*80}")
else:
    print(f"\n⏭️  OPTIMIZE SKIPPED: Conditions not met")
    print(f"  - Docs processed: {doc_count} (threshold: >100)")
    print(f"  - Files: {num_files} (threshold: >200 with small files OR >500 total)")
    print(f"  - Avg file size: {avg_file_size_mb:.2f}MB (threshold: <100MB)")
    print(f"{'='*80}")
